<a href="https://colab.research.google.com/github/misrori/trading_strategy/blob/main/rsi_base_strategy_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/misrori/binance_data
!pip install goldhand
!pip install numpy==1.26.4
exit()

Cloning into 'binance_data'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 136 (delta 0), reused 0 (delta 0), pack-reused 135 (from 3)
Receiving objects: 100% (136/136), 312.18 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Updating files: 100% (106/106), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 7.6 MB/s eta 0:00:00
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=c3476cf794a1c68b11926035baede80eb8352cb8901be2511a5b4a4b43b03132
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
  Attempting

In [1]:
import pandas as pd
import pandas_ta as ta
import os
import warnings
import pickle
warnings.filterwarnings('ignore')

def smma(data, window, colname):
    hl2 = data['hl2'].values
    smma_values = [hl2[0]]

    for i in range(1, len(hl2)):
        smma_val = (smma_values[-1] * (window - 1) + hl2[i]) / window
        smma_values.append(smma_val)

    data[colname] = smma_values
    return data


def read_one_file(fpath):

    #t = "/content/binance_startegy/data/BTCUSDT_1h.csv"
    # read pickle
    with open(fpath, 'rb') as f:
        df = pickle.load(f)
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    df['date'] = pd.to_datetime(df['date'])
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)

    # Rsi
    df['rsi'] = ta.rsi(df['close'], 14)

    # SMAS
    df['sma_50']= ta.sma(df['close'], 50)
    df['diff_sma50'] = (df['close']/df['sma_50'] -1)*100
    df['sma_100']= ta.sma(df['close'], 100)
    df['diff_sma100'] = (df['close']/df['sma_100'] -1)*100
    df['sma_200']= ta.sma(df['close'], 200)
    df['diff_sma200'] = (df['close']/df['sma_200'] -1)*100


    # EMAS
    df['ema_7']= ta.ema(df['close'], 7)
    df['diff_ema7'] = (df['close']/df['ema_7'] -1)*100
    df['ema_14']= ta.ema(df['close'], 14)
    df['diff_ema14'] = (df['close']/df['ema_14'] -1)*100
    df['ema_21']= ta.ema(df['close'], 21)
    df['diff_ema21'] = (df['close']/df['ema_21'] -1)*100

    #Bolinger bands
    bb = ta.bbands(df['close'])
    bb.columns = ['bb_lower', 'bb_mid', 'bb_upper', 'bandwidth', 'percent']
    df['bb_lower'] = bb['bb_lower']
    df['bb_upper'] = bb['bb_upper']
    df['diff_upper_bb'] = (df['bb_upper']/df['close'] -1)*100
    df['diff_lower_bb'] = (df['bb_lower']/df['close'] -1)*100


    # Apply SMMA to the dataframe
    df['hl2'] = (df['high'] + df['low'])/2
    df = smma(df, 7, 'v1')
    df = smma(df, 11, 'v2')
    df = smma(df, 16, 'v3')
    df = smma(df, 21, 'v4')
    ticker = str(fpath).replace('/content/binance_data/data/USDT/', '').replace('USDT_binance_5min_data.pkl', '')
    df['ticker'] = ticker

    return df

# list files
all_file = os.listdir('/content/binance_data/data/USDT',)
all_file = ['/content/binance_data/data/USDT/'+x for x in all_file]


In [3]:

def rsi_strategy(data, buy_threshold = 30, sell_threshold = 70):
    """
    RSI strategy for backtesting with Backtest class

    Parameters:
    - data: pandas DataFrame with columns: date, open, high, low, close, volume and rsi
    - buy_threshold: int, default 30,  buy when RSI is below this value
    - sell_threshold: int, default 70, sell when RSI is above this value
    """

    in_trade = False  # Flag to track if already in a trade
    trade_id = 1
    all_trades = []

    temp_trade = {}

    for i in range(1, len(data)):
        # Check if not already in a trade
        if not in_trade:
            # Generate buy signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] < buy_threshold:

                if i == (len(data) -1):  # this is if the last wor have the signal there is no i+1
                    temp_trade['buy_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('buy_')))
                else:
                    temp_trade['buy_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('buy_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'open'
                in_trade = True  # Set flag to indicate in a trade
        else:
            # Generate sell signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] > sell_threshold:
                if i == (len(data) -1):  # this is if the last wor have the signal there is no i+1
                    temp_trade['sell_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('sell_')))
                else:
                    temp_trade['sell_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('sell_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'closed'


                # calculate results
                temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
                temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days

                in_trade = False  # Reset flag to indicate not in a trade
                trade_id +=1
                all_trades.append(temp_trade)
                temp_trade = {}

    if temp_trade:
        temp_trade['sell_price'] = data['close'][i]
        temp_trade['trade_id'] = trade_id
        temp_trade['sell_date'] = data['date'][i]

        temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
        temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days
        all_trades.append(temp_trade)

    res_df = pd.DataFrame(all_trades)

    # change orders
    all_col = res_df.columns.tolist()
    first = ['result', 'buy_price', 'sell_price', 'buy_date', 'sell_date', 'days_in_trade']
    first.extend([x for x in all_col if x not in first])
    res_df = res_df[first]
    return(res_df)


In [13]:
from IPython.display import Javascript
from goldhand import *
from tqdm import tqdm
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 8000})'''))

symbol = 'BTC'

data = read_one_file(f'/content/binance_data/all_timeframe/BTCUSDT/BTCUSDT_binance_4h_data.pkl')

backtest = Backtest( data, rsi_strategy, buy_threshold=25, sell_threshold=80)

backtest.trades_summary



<IPython.core.display.Javascript object>

{'ticker': '/content/binance_data/all_timeframe/BTCUSDT/BTCUSDT_binance_4h_data.pkl',
 'number_of_trades': 17,
 'win_ratio(%)': 58.82,
 'average_res(%)': 4.18,
 'average_trade_len(days)': 38.0,
 'median_res(%)': 7.59,
 'cumulative_result': '1.59 x',
 'trade_results': '-27.01 # 22.13 # -26.49 # -18.31 # -8.31 # 11.58 # 24.16 # -0.14 # 7.59 # -3.13 # 11.99 # 15.67 # 12.63 # -2.3 # 34.35 # 14.14 # 2.58',
 'profitable_trade_results': '22.13 # 11.58 # 24.16 # 7.59 # 11.99 # 15.67 # 12.63 # 34.35 # 14.14 # 2.58',
 'profitable_trades_mean': 15.68,
 'profitable_trades_median': 13.38,
 'looser_trade_results': '-27.01 # -26.49 # -18.31 # -8.31 # -0.14 # -3.13 # -2.3',
 'looser_trades_mean': -12.24,
 'looser_trades_median': -8.31,
 'median_trade_len(days)': 34.0,
 'number_of_win_trades': 10,
 'number_of_lost_trades': 7,
 'max_gain(%)': 34.35,
 'max_lost(%)': -27.01,
 'first_trade_buy': Timestamp('2021-11-17 00:00:00'),
 'first_data_date': '2021-11-10',
 'first_open_price': 66947.67,
 'last_data_d

In [14]:
backtest.trades


,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_high,buy_low,...,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_hl2,sell_v1,sell_v2,sell_v3,sell_v4,sell_ticker
0,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.729877,60058.87,43835.59,2021-11-17 00:00:00,2022-02-07 16:00:00,82,60058.87,60437.68,58553.12,...,41977.248468,44563.283532,0.701613,-5.142164,43992.705,42170.934234,41457.973252,40776.735262,40285.173891,/content/binance_data/all_timeframe/BTCUSDT/BT...
1,/content/binance_data/all_timeframe/BTCUSDT/BT...,1.221254,38343.99,46827.76,2022-02-20 12:00:00,2022-03-28 00:00:00,35,38343.99,38616.00,38022.00,...,43471.118386,47782.817614,1.558825,-7.605371,47156.780,45011.309161,44564.133417,44098.964623,43699.799857,/content/binance_data/all_timeframe/BTCUSDT/BT...
2,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.735091,43170.47,31734.23,2022-04-07 00:00:00,2022-05-31 00:00:00,54,43170.47,43559.09,42727.35,...,30005.799385,32174.040615,1.262779,-5.561422,31651.035,30312.113972,29981.271752,29799.110636,29729.760891,/content/binance_data/all_timeframe/BTCUSDT/BT...
3,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.816874,25520.05,20846.66,2022-06-13 04:00:00,2022-10-26 16:00:00,135,25520.05,26257.31,24555.00,...,20100.140277,21025.435723,1.261325,-3.195022,20818.555,20081.299820,19854.686688,19695.337573,19601.784264,/content/binance_data/all_timeframe/BTCUSDT/BT...
4,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.916889,19699.04,18061.83,2022-11-08 12:00:00,2022-12-14 16:00:00,36,19699.04,19746.78,19229.00,...,17643.130462,18063.809538,1.522571,-0.841738,18048.785,17649.147206,17505.625018,17396.867573,17322.842413,/content/binance_data/all_timeframe/BTCUSDT/BT...
5,/content/binance_data/all_timeframe/BTCUSDT/BT...,1.115839,21797.83,24322.87,2023-02-10 00:00:00,2023-02-16 00:00:00,6,21797.83,21938.16,21756.00,...,21778.552054,25273.431946,2.187461,-11.943303,24591.790,22762.378181,22486.255928,22367.510764,22349.647746,/content/binance_data/all_timeframe/BTCUSDT/BT...
6,/content/binance_data/all_timeframe/BTCUSDT/BT...,1.241575,20856.30,25894.67,2023-03-09 20:00:00,2023-03-14 16:00:00,4,20856.30,20867.30,20042.72,...,23685.331067,26040.148933,3.935472,-5.463442,25107.150,23604.256451,22910.544185,22483.271254,22300.914122,/content/binance_data/all_timeframe/BTCUSDT/BT...
7,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.998599,28104.38,28065.01,2023-04-20 20:00:00,2023-05-29 00:00:00,38,28104.38,28365.50,28012.00,...,26870.964988,28317.799012,1.297833,-3.877751,28180.710,27265.145304,27082.639446,26983.611757,26946.058706,/content/binance_data/all_timeframe/BTCUSDT/BT...
8,/content/binance_data/all_timeframe/BTCUSDT/BT...,1.075949,26007.53,27982.77,2023-06-05 16:00:00,2023-06-20 20:00:00,15,26007.53,26074.51,25388.00,...,26209.420273,28631.827727,1.143980,-7.413348,28137.370,26991.356254,26769.091871,26591.205703,26484.077209,/content/binance_data/all_timeframe/BTCUSDT/BT...
9,/content/binance_data/all_timeframe/BTCUSDT/BT...,0.968703,28730.51,27831.33,2023-08-17 00:00:00,2023-08-29 20:00:00,12,28730.51,28783.48,28342.00,...,25335.074059,28645.269941,3.351561,-8.591560,27708.200,26575.648094,26406.880249,26335.129708,26345.007108,/content/binance_data/all_timeframe/BTCUSDT/BT...


In [16]:
backtest.show_trades()